In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import nbformat

# Path to your current notebook (change this if needed)
input_path = '/content/drive/MyDrive/Colab_Notebooks/pred_next_word/pred_next_word.ipynb'
output_path = '/content/drive/MyDrive/Colab_Notebooks/pred_next_word/pred_next_wyour_notebook_clean.ipynb'

# Load and clean the notebook
with open(input_path, 'r') as f:
    nb = nbformat.read(f, as_version=4)

# Remove widget metadata if present
if 'widgets' in nb['metadata']:
    del nb['metadata']['widgets']

# Save the cleaned notebook
with open(output_path, 'w') as f:
    nbformat.write(nb, f)

print(f"Clean version saved to {output_path}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/pred_next_word.ipynb'

In [ ]:
import string
import nltk
import re
import string
from nltk.corpus import stopwords
import pkg_resources
import pickle
import nltk
import re, string, json
from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize

<ipython-input-4-9726596cf16c>:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [ ]:
from tensorflow import keras
from numpy import array
from pickle import dump
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

from random import randint
from pickle import load
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
def load_doc(filename):
  file=open(filename,'r')
  text=file.read()
  file.close()
  return text


In [ ]:
def tokenize_twitter(sentences):
  tokenized_sentences=[]
  for sentence in tqdm(sentences):
    sentence=cleanhtml(sentence)
    sentence=_replace_urls(sentence)
    sentence=remove_email(sentence)
    sentence=re.sub(r'[^a-zA-Z]',' ',sentence)
    sentence=sentence.lower()
    sentence=misc(sentence)
    tokenized_sentences.append(sentence)
  return tokenized_sentences

In [ ]:
def cleanhtml(raw_html):
  cleanr=re.compile('<.*?>')
  cleantext=re.sub(cleanr,'',raw_html)
  return cleantext

def _replace_urls(data):
  url_pattern=re.compile(r'https?://\S+|www\.\S+')
  data=url_pattern.sub(r'',data)
  return data

def remove_email(data):
  data = re.sub('\S*@\S*\s?', '', data)
  return data

def misc(data):
  data = re.sub('\s+',' ', data)
  data = re.sub("\'", "", data)
  data = re.sub("ww+", "", data)
  MAYBE_ROMAN = re.compile(r'(\b[MDCLXVI]+\b)(\.)?', re.I)
  data = re.sub (MAYBE_ROMAN, "", data)
  return data

In [ ]:
def littleCleaning (sentences):
  print("Starting cleaning Process")
  ret_list = []
  for sentence in sentences:
    words =sentence.split(" ")
    if len(words) > 5:
      ret_list.append(sentence)
    else:
      continue
  return ret_list

In [ ]:
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:

path = '/content/drive/MyDrive/Colab_Notebooks/pred_next_word/republic.txt'
text=open(path).read().lower()
print('length is : :', len(text))


length is : : 388072


In [ ]:
data_list = text.split(".")
data_list

['\ufeff    \nthis ebook is for the use of anyone anywhere in the united states and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever',
 ' you may copy it, give it away or re-use it under the terms\nof the project gutenberg license included with this ebook or online\nat www',
 'gutenberg',
 'org',
 ' if you are not located in the united states,\nyou will have to check the laws of the country where you are located\nbefore using this ebook',
 '\n\n\nmost of the adventures recorded in this book really occurred; one or two\nwere experiences of my own, the rest those of boys who were schoolmates\nof mine',
 ' huck finn is drawn from life; tom sawyer also, but not from an\nindividual—he is a combination of the characteristics of three boys whom\ni knew, and therefore belongs to the composite order of architecture',
 '\n\nthe odd superstitions touched upon were all prevalent among children and\nslaves in the west at the period of this story—that is to say, 

In [ ]:
pro_sentences = []

def normalization_pipeline(sentences):
  print("start normalization")
  sentences = tokenize_twitter(sentences)
  sentences = littleCleaning(sentences)
  print("normalization done")
  return sentences

pro_sentences = normalization_pipeline(data_list)
pro_sentences[:5]


start normalization


  0%|          | 0/3899 [00:00<?, ?it/s]

Starting cleaning Process
normalization done


[' this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with almost no restrictions whatsoever',
 ' you may copy it give it away or re use it under the terms of the project gutenberg license included with this ebook or online at ',
 ' if you are not located in the united states you will have to check the laws of the country where you are located before using this ebook',
 ' most of the adventures recorded in this book really occurred one or two were experiences of my own the rest those of boys who were schoolmates of mine',
 ' huck finn is drawn from life tom sawyer also but not from an individual he is a combination of the characteristics of three boys whom  knew and therefore belongs to the composite order of architecture']

In [ ]:
len(pro_sentences)

3521

In [ ]:
dataText = "".join(pro_sentences[: 700])
dataText[:200]

' this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with almost no restrictions whatsoever you may copy it give it away or re use it und'

In [ ]:
def clean_doc(doc):
  doc=doc.replace('--',' ')
  tokens = doc.split()
  table=str.maketrans('', '',string.punctuation)
  tokens=[w.translate(table) for w in tokens]
  tokens=[word for word in tokens if word.isalpha()]
  tokens=[word.lower() for word in tokens]
  return tokens


In [ ]:
tokens = clean_doc(dataText)
print(tokens[:200])
print('total tokens:%d' %len(tokens))
print('unique tokens:%d' %len(set(tokens)))

['this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'in', 'the', 'united', 'states', 'and', 'most', 'other', 'parts', 'of', 'the', 'world', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 're', 'use', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'if', 'you', 'are', 'not', 'located', 'in', 'the', 'united', 'states', 'you', 'will', 'have', 'to', 'check', 'the', 'laws', 'of', 'the', 'country', 'where', 'you', 'are', 'located', 'before', 'using', 'this', 'ebook', 'most', 'of', 'the', 'adventures', 'recorded', 'in', 'this', 'book', 'really', 'occurred', 'one', 'or', 'two', 'were', 'experiences', 'of', 'my', 'own', 'the', 'rest', 'those', 'of', 'boys', 'who', 'were', 'schoolmates', 'of', 'mine', 'huck', 'finn', 'is', 'drawn', 'from', 'life', 'tom', 'sawyer', 'also', 'but', 'not', 'from', 'an'

In [ ]:
length = 5+1
sequences = list()
for i in range(length,len(tokens)):
  seq=tokens[i-length:i]
  line=' '.join(seq)
  sequences.append(line)
print('total sequences:%d' %len(sequences))


total sequences:15741


In [ ]:
def save_doc(lines,filename):
  data='\n'.join(lines)
  file=open(filename,'w')
  file.write(data)
  file.close()

out_filename='/content/drive/MyDrive/Colab_Notebooks/pred_next_word/republic_sequences.txt'
save_doc(sequences,out_filename)

In [ ]:
in_filename='/content/drive/MyDrive/Colab_Notebooks/pred_next_word/republic_sequences.txt'
doc=load_doc(in_filename)
lines=doc.split('\n')

tokenizer=Tokenizer()
tokenizer.fit_on_texts(lines)
sequences=tokenizer.texts_to_sequences(lines)
vocab_size=len(tokenizer.word_index)+1
sequences=array(sequences)
X,y=sequences[:,:-1],sequences[:,-1]
y=to_categorical(y,num_classes=vocab_size)
seq_length=X.shape[1]

In [ ]:
model=Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,batch_size=128,epochs=100)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0229 - loss: 7.4868
Epoch 2/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0508 - loss: 6.2981
Epoch 3/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0500 - loss: 6.2805
Epoch 4/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.0519 - loss: 6.2129
Epoch 5/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.0517 - loss: 6.0944
Epoch 6/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0527 - loss: 6.0207
Epoch 7/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0635 - loss: 5.9260
Epoch 8/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0657 - loss: 5.8273
Epoch 9/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0656 - loss: 5.7139
Epoch 10/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0724 - loss: 5.6147
Epoch 11/100
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0727 - loss: 5.5459
Epoch 12/100
123/123 ━━━━━━━━━━━━━

In [ ]:
model.save('/content/drive/MyDrive/Colab_Notebooks/pred_next_word/model.h5')

In [ ]:
dump(tokenizer,open('/content/drive/MyDrive/Colab_Notebooks/pred_next_word/tokenizer.pkl','wb'))

In [ ]:
import numpy as np

def generate_seq(model,tokenizer,seq_length,seed_text,n_words):
  result=list()
  in_text=seed_text
  for _ in range(n_words):
    encoded=tokenizer.texts_to_sequences([in_text])[0]
    encoded=pad_sequences([encoded],maxlen=seq_length,truncating='pre')
    #yhat=model.predict_classes(encoded)
    predict_x=model.predict(encoded)
    yhat=np.argmax(predict_x,axis=1)
    out_word=''
    for word,index in tokenizer.word_index.items():
      if index==yhat:
        out_word=word
        break
    in_text+=' '+out_word
    result.append(out_word)
  return ' '.join(result)


In [ ]:
in_filename='/content/drive/MyDrive/Colab_Notebooks/pred_next_word/republic_sequences.txt'
doc=load_doc(in_filename)
lines=doc.split('\n')
seq_length=len(lines[0].split())-1

print(len(lines))
print(lines[0])

15741
this ebook is for the use


In [ ]:
model=load_model('/content/drive/MyDrive/Colab_Notebooks/pred_next_word/model.h5')
tokenizer=load(open('/content/drive/MyDrive/Colab_Notebooks/pred_next_word/tokenizer.pkl','rb'))
seed_text=lines[randint(0,len(lines))]
print(seed_text + '\n')
generated=generate_seq(model,tokenizer,seq_length,seed_text,12)
print(generated)

that s what it is forty

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
times how get it see what tricks hookey of the window and


In [ ]:
tt="To which Tom responded with"
seq_length=len(tt.split())

generated=generate_seq(model,tokenizer,seq_length,tt,1)
print(generated)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
jeers
